In [1]:
import numpy as np

In [2]:
class Dynamics:
    pass

In [3]:
hw5dynamics=Dynamics()
hw5dynamics.kappa = 0.472
hw5dynamics.alpha = 4.4
hw5dynamics.sigma = 0.368
hw5dynamics.S0 = 106.9
hw5dynamics.r = 0.05

In [4]:
class Contract:
    pass

In [5]:
hw5contract=Contract()
hw5contract.K1 = 103.2
hw5contract.T1 = 0.5
hw5contract.T2 = 0.75

In [8]:
class MC:
    pass

In [9]:
hw5MC=MC()
hw5MC.N = 100   # Number of timesteps on each path
hw5MC.M = 100000  # Number of paths.  Change this if necessary.
hw5MC.epsilon = 0.01  # For the dC/dS calculation
hw5MC.seed = 0  # Seeding the random number generator with a specified number helps make the calculations reproducible

In [25]:
def pricer_oilcall_OU_MC(contract,dynamics,MC):
    
    K1,T1,T2 = contract.K1,contract.T1,contract.T2
    kappa,alpha,sigma,S0,r = dynamics.kappa,dynamics.alpha,dynamics.sigma,dynamics.S0,dynamics.r
    N,M,epsilon = MC.N,MC.M,MC.epsilon

    np.random.seed(MC.seed)  #seed the random number generator
    
    deltat = T1/N
    t = np.linspace(0,T1,num=N+1)
    S0_delta = S0+epsilon
    
    def path(S0,dWt):
        X0 = np.log(S0)
        Xt = [X0]
        for i in range(N):
            currX = Xt[i] + kappa*(alpha-Xt[i])*deltat+sigma*dWt[i] ## simulate the path from X0 to Xt, using Euler Method
            Xt.append(currX)
        Ft = np.exp(np.exp(-kappa*(T2-t))*Xt + alpha*(1-np.exp(-kappa*(T2-t))) + (sigma**2)/(4*kappa)*(1-np.exp(-2*kappa*(T2-t))))
        callpayoff = np.exp(-r*T1)*np.maximum(Ft[-1]-K1,0) ## get the payoff at expiry time T
        return callpayoff
    
    payoffs1 = []
    payoffs2 = []
    for _ in range(M):
        dWt = np.sqrt(deltat)*np.random.normal(0,1,size=N)
        payoff1 = path(S0,dWt)
        payoff2 = path(S0_delta,dWt)
        payoffs1.append(payoff1)  ## get M amount of payoff
        payoffs2.append(payoff2)
    
    call_price = np.average(payoffs1)  ## average will be the final payoff
    standard_error = np.std(payoffs1)/np.sqrt(M)
    call_delta = (np.average(payoffs2)-call_price)/epsilon
    return (call_price,standard_error,call_delta)

        

In [26]:
(call_price, standard_error, call_delta) = pricer_oilcall_OU_MC(hw5contract,hw5dynamics,hw5MC)

In [27]:
print(call_price, standard_error, call_delta)

7.821380965248459 0.04194572483525262 0.34443701918993597


In [19]:
#### 1e

S0 = hw5dynamics.S0
r = hw5dynamics.r
T2 = hw5contract.T2
kappa = hw5dynamics.kappa
alpha = hw5dynamics.alpha
sigma = hw5dynamics.sigma

S0**(-1+np.exp(-kappa*T2))*np.exp(-r*T2-kappa*T2+(1-np.exp(-kappa*T2))*alpha+sigma**2/(4*kappa)*(1-np.exp(-2*kappa*T2)))

0.6465109164314246

In [20]:
#### 1f


call_delta/(S0**(-1+np.exp(-kappa*T2))*np.exp(-r*T2-kappa*T2+(1-np.exp(-kappa*T2))*alpha+sigma**2/(4*kappa)*(1-np.exp(-2*kappa*T2))))

0.5327628821662294

In [21]:
#### 1g

K1 = hw5contract.K1
T1 = hw5contract.T1

disc = np.exp(-kappa*T2)
F0 = np.exp(disc*(np.log(S0))+(1-disc)*alpha+sigma**2/(4*kappa)*(1-disc**2))
f0 = (F0-K1)*np.exp(-r*T2)
v0 = 4000*f0+1000*call_price*np.exp(-r*(T2-T1))
v0

3988.3850598670606